<a href="https://colab.research.google.com/github/GuyRobot/AINotesBook/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q huggingface_hub datasets evaluate transformers rouge-score nltk transformers[torch] sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 29.5 MB/s eta 0:00:00


In [2]:
!pip install -q kaggle
!mkdir ~/.kaggle
!wget https://raw.githubusercontent.com/GuyRobot/AINotesBook/main/kaggle.json
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download nguyentrungtam/ehealthvnchatdataset
!kaggle datasets download nguyentrungtam/ehealthchatdataset
!mkdir -p /kaggle/input
!mv ehealthvnchatdataset.zip /kaggle/input/
!cd /kaggle/input/ && unzip ehealthvnchatdataset.zip -d ehealthvnchatdataset

!mv ehealthchatdataset.zip /kaggle/input/
!cd /kaggle/input/ && unzip ehealthchatdataset.zip -d ehealthchatdataset

--2023-11-23 00:39:11--  https://raw.githubusercontent.com/GuyRobot/AINotesBook/main/kaggle.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70 [text/plain]
Saving to: ‘kaggle.json’

kaggle.json         100%[===================>]      70  --.-KB/s    in 0s      

2023-11-23 00:39:12 (647 KB/s) - ‘kaggle.json’ saved [70/70]

 99% 61.0M/61.3M [00:04<00:00, 18.5MB/s]
100% 61.3M/61.3M [00:04<00:00, 13.9MB/s]
100% 586M/588M [00:33<00:00, 19.5MB/s]
100% 588M/588M [00:33<00:00, 18.4MB/s]
Archive:  ehealthvnchatdataset.zip
  inflating: ehealthvnchatdataset/EHealthVNChatDataCrawled.csv  
  inflating: ehealthvnchatdataset/EHealthVNChatDataset.csv  
Archive:  ehealthchatdataset.zip
  inflating: ehealthchatdataset/EHealthChatDataset.csv  
  inflating: ehealthchatdat

In [3]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained("GuysTrans/bart-base-finetuned-xsum", revision="f98771cfa83c1cdf8b2cce100922eaf36420abcd")

config.json:   0%|          | 0.00/1.75k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

In [8]:
import datasets
from datasets import load_dataset
from evaluate import load
import json
import pandas as pd
from datasets import Dataset

In [25]:
with open("/kaggle/input/ehealthvnchatdataset/EHealthVNChatDataCrawled.csv") as f:
    data = pd.read_csv(f)
    data = data.filter(['question', 'answer'])
    data = data.dropna()
    train_ds = Dataset.from_pandas(pd.DataFrame(data=data))
    vals_ds = Dataset.from_pandas(pd.DataFrame(data=data[-10000:]))

In [26]:
len(data.loc[(data["answer"].str.len() > 512)])

93851

In [27]:
min(data["question"].str.len())

1

In [28]:
max(data["question"].str.len())

264

In [32]:
sum(data["answer"].str.len()) / len(data)

693.3277221185589

In [30]:
# def generate_summary(question, model):
#     inputs = tokenizer(
#         question,
#         padding="max_length",
#         truncation=True,
#         max_length=512,
#         return_tensors="pt",
#     )
#     input_ids = inputs.input_ids.to(model.device)
#     attention_mask = inputs.attention_mask.to(model.device)
#     outputs = model.generate(
#         input_ids, attention_mask=attention_mask, max_new_tokens=4096, do_sample=True, num_beams=4, top_k=50, early_stopping=True, no_repeat_ngram_size=2)
#     output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     return outputs, output_str

In [31]:
# generate_summary("Dấu hiệu nhận biết bệnh lậu", model)[1]